In [16]:
from datasets import load_dataset
import spacy

from pathlib import Path
from os import listdir
from os.path import isfile, join
import os

In [17]:
# download stories from here
# https://cs.nyu.edu/~kcho/DMQA/

# 114 out of over 300,000 stories are missing text

# makes datafiles from this code
# https://github.com/becxer/cnn-dailymail/blob/master/make_datafiles.py

In [18]:
# """Load the dataset using huggingface data_loader"""
# type this command in the command line for the venv for this cell to work
# jupyter nbextension enable --py widgetsnbextension --sys-prefix 
# dataset = load_dataset('cnn_dailymail', '3.0.0')

In [19]:
CNN_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/cnn/stories")
DM_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/dailymail/stories")

In [21]:
def grab_files(directory):
    for name in os.listdir(directory):
        full_path = os.path.join(directory, name)
        if os.path.isdir(full_path):
            for entry in grab_files(full_path):
                yield entry
        elif os.path.isfile(full_path):
            yield full_path
        else:
            print('Unidentified name %s. It could be a symbolic link' % full_path)

In [22]:
i = 0
for file in grab_files(CNN_STORIES_PATH):
    i+=1
    print(file)
    if i == 5:
        break

D:\__School\__Masters\____2021fALL\5290 nlp\stories\cnn\stories\0001d1afc246a7964130f43ae940af6bc6c57f01.story
D:\__School\__Masters\____2021fALL\5290 nlp\stories\cnn\stories\0002095e55fcbd3a2f366d9bf92a95433dc305ef.story
D:\__School\__Masters\____2021fALL\5290 nlp\stories\cnn\stories\00027e965c8264c35cc1bc55556db388da82b07f.story
D:\__School\__Masters\____2021fALL\5290 nlp\stories\cnn\stories\0002c17436637c4fe1837c935c04de47adb18e9a.story
D:\__School\__Masters\____2021fALL\5290 nlp\stories\cnn\stories\0003ad6ef0c37534f80b55b4235108024b407f0b.story


In [24]:
f = open(CNN_STORIES_PATH.joinpath("0001d1afc246a7964130f43ae940af6bc6c57f01.story"), "r")
print(f.read())

It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria.

Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons.

The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction."

It's a step that is set to turn an international crisis into a fierce domestic political battle.

There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react?

In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he wa

In [25]:
f = open(CNN_STORIES_PATH.joinpath("0002095e55fcbd3a2f366d9bf92a95433dc305ef.story"), "r")
print(f.read())

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.

The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.

The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.

The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles.

The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital.

"I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics.

Victory was never 